In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import credentials
from pyairtable import Table
import folium

SETUP

In [ ]:
# this is just to get rid of an annoying warning that doesn't actually apply to this code
pd.options.mode.chained_assignment = None

# pull data from airtable
api_key = credentials.AIRTABLE_API_KEY

base_id = 'appK4hPIw3yecviBP'
table_id = 'tblUhVjXhOKsYqd3x'
# this is the fossil fuel plant only view
view_id = 'viwCiVUfX5kemiT2z'
fields = ['Plant Code', 'Generators from 1982 or older', 'Ownership']

table = Table(api_key, base_id, table_id)

# read EIA data and US map
plants_location = "shapefiles/Plants/PowerPlants_US_202108.shp"
US_map_location = "shapefiles/US/cb_2018_us_nation_20m.shp"

# read files into data frames
plants = gpd.read_file(plants_location)
US_map = gpd.read_file(US_map_location)

DATA GATHERING

In [ ]:
# create new columns in EIA data frames to insert airtable data into
plants['old_generators'] = 0
plants['ownership'] = ""

# insert airtable data into EIA data frames
for page in table.iterate(view = view_id, fields = fields):
    for airtable_plant in page:
        # store field values from airtable data
        fields = airtable_plant['fields']
        plant_code = fields['Plant Code']
        old_generators = fields['Generators from 1982 or older']
        ownership = fields['Ownership']

        # locate matching record in EIA data and set old_generators to correct value
        plants.loc[plants['Plant_Code'] == plant_code, ['old_generators']] = old_generators
        plants.loc[plants['Plant_Code'] == plant_code, ['ownership']] = ownership

DATA MODIFICATION

In [ ]:
# filter out unwanted plants
target_plants = plants[(plants.old_generators > 0) & (plants.ownership == 'S')]

# normalize plant size
scale = 20
base = 5

largest_plant_size = target_plants['Total_MW'].max()
smallest_plant_size = target_plants['Total_MW'].min()
plant_size_range = largest_plant_size - smallest_plant_size
target_plants['norm_plant_size'] = (target_plants.Total_MW - smallest_plant_size) / plant_size_range * scale + base

# create color buckets
# these are assigned numbers because folium requires each point be added individually, so a color map cannot be used
# colors are assigned when the map is created
target_plants['old_generator_bucket'] = 0

target_plants.loc[(target_plants['old_generators'] > 0) & (target_plants['old_generators'] <= 4), ['old_generator_bucket']] = 1
target_plants.loc[(target_plants['old_generators'] > 4) & (target_plants['old_generators'] <= 8), ['old_generator_bucket']] = 2
target_plants.loc[(target_plants['old_generators'] > 8) & (target_plants['old_generators'] <= 12), ['old_generator_bucket']] = 3
target_plants.loc[(target_plants['old_generators'] > 12) & (target_plants['old_generators'] <= 20), ['old_generator_bucket']] = 4

DEBUGGING

In [ ]:
# # print the columns in terminal
# pd.set_option("display.max_rows", None, 'display.max_columns',None)
# print(target_plants.head())

PLOTTING

In [ ]:
# make the map
m = folium.Map(location=[39, -98], zoom_start=5)

# iterate through all the rows to add each plant to the map individually
for row in target_plants.itertuples():
   # grab and format tooltip/popup information
   plant_name = "Plant name: " + row.Plant_Name
   utility_name = "Utility name: " + row.Utility_Na
   city = "City: " + row.City
   state = "State: " + row.StateName
   html = plant_name + "<br>" + utility_name + "<br>" + city + "<br>" + state

   # assign colors
   color = ""
   if row.old_generator_bucket == 1:
      color = '#440154'
   if row.old_generator_bucket == 2:
      color = '#31688e'
   if row.old_generator_bucket == 3:
      color = '#35b779'
   if row.old_generator_bucket == 4:
      color = '#fde725'

   # set up popup
   iframe = folium.IFrame(html)
   popup = folium.Popup(iframe,
                     min_width=300,
                     max_width=300)

   # generate icons
   size=row.norm_plant_size
   icon=folium.DivIcon(html=f"""
                        <div><svg>
                           <circle cx="{size}" cy="{size}" r="{size}" fill="{color}" opacity=".7"/>
                        </svg></div>""")

   # add each marker individually
   folium.Marker(
      location=[row.Latitude, row.Longitude],
      tooltip=plant_name,
      popup=popup,
      icon=icon,
   ).add_to(m)

m

SAVING

In [193]:
m.save("dynamic_map.html")